In [ ]:
import time
import sqlite3
import requests
import datetime
import pandas as pd
import shioaji as sj
import pickle


from exchange_calendars import get_calendar
import mplfinance as mpf
#import talib

from FuncBase import *

In [ ]:
import os 
import shioaji as sj
from dotenv import load_dotenv

#永豐金API登入
load_dotenv()
api = sj.Shioaji(simulation=True)
api.login(os.getenv('API_KEY'),os.getenv('SECRET_KEY'),  
          contracts_cb= lambda security_type: print(f"{repr(security_type)} fetch done"))

#連結DB Browser
connection = sqlite3.connect('work_data.db')

In [ ]:
from shioaji import TickSTKv1, Exchange
from shioaji import TickFOPv1, Exchange

#設定callback 回傳資料型態
@api.on_tick_stk_v1()
def stock_quote_callback(exchange: Exchange, tick:TickSTKv1):

    #full_code = msg['content'][0]  # 假設這是完整的商品代號
    #code = full_code[:3] 

    ts = pd.to_datetime(tick.datetime, format='%Y-%m-%d %H:%M:%S')

    #f tick.code[0].isalpha():
    #    code = tick.code[0:3]
    #else:
    code = tick.code

    ts = pd.to_datetime(tick.datetime)


    if 5 < ts.hour < 9:
        return
    
    
    close = float(tick.close)
    volume = tick.volume
    vol_sum = tick.total_volume

    #print(close)
    #print(volume)
    #print(vol_sum)

    # 確保代號在 ticks 中存在
    #if code not in ticks:
    #    ticks[code] = pd.DataFrame(columns=["close", "volume"])

    # 更新 ticks 和 volume_today 資料
    ticks[code].loc[ts] = [close, volume]
    volume_today[code] = vol_sum
    print(ticks[code].tail(10))
    #print(volume_today)

@api.on_tick_fop_v1()
def fut_quote_callback(exchange: Exchange, tick: TickFOPv1):

    ts = pd.to_datetime(tick.datetime)

    if tick.code[0].isalpha():
        tick.datetime
        code = tick.code[0:3]
    else:
        #print('tick.code is not only str')
        code = tick.code
        

    ts = pd.to_datetime(tick.datetime)

    if 5 < ts.hour < 9:
        return
    
    close = float(tick.close)
    volume = tick.volume
    vol_sum = tick.total_volume

    
    ticks[code].loc[ts] = [close, volume]
    volume_today[code] = vol_sum


def place_cb(stat, msg):

    if stat.value == 'TFTDEAL':

        code = msg['code']
        qty = int(msg['quantity'])
                
        order_qty[code] -= qty #扣除下單時委託張數

        if order_qty[code] == 0:
            open_orders = None #委託張數歸0

        position = positions.get(code,0)

        if msg['action'] == 'Sell':
            qty = qty * -1

        position += qty

        positions[code] = position
    
api.set_order_callback(place_cb)


In [ ]:
ticks = {}
kbars = {}
volume_today = {}

open_orders = {}
order_qty = {}
positions = {}

workers = {}
stop_threads = False


In [ ]:
import time
import pickle

def M_backtest(connection, api, start_date, end_date,
               codes, is_Futures = False,
               filter = 0.08,
               Ratio_Drawdown1 = -0.05,
               Ratio_Drawdown2 = -0.03,
               Quit_position = 0
               ):

    print('--- 啟動 Tbot for {} ---'.format(codes))

    tw_calendar = get_calendar('XTAI')

    #Risk Variables
    filter = filter
    Ratio_Drawdown1 = Ratio_Drawdown1 #預設台股最大漲跌幅
    Ratio_Drawdown2 = Ratio_Drawdown2
    Quit_position = Quit_position

    #Situation Variable
    entry_time = None
    entry_price = 0

    exit_time = None
    exit_price = 0

    position = 0
    add_position = 0
    last_price = 0
    #平均成本
    AC = 0
    #Entry High
    EH = 0

    transactions = pd.DataFrame(columns=[
        '成交日期',
        '股票代號',
        '進場時間',
        '平均買進價格',
        '總買進口數',
        '賣出時間',
        '賣出價格',
        '總賣出口數',
        '賣出價格',
        '價差'
    ])

    #Get historical data
    all_kbar = historical_kbars(connection, api, start_date, end_date,
                      '30Min', codes, is_Futures)
    all_kbar = get_MA(all_kbar)
    all_kbar.index = pd.to_datetime(all_kbar.index)


    for date in pd.date_range(start_date, end_date):
        
        if date not in tw_calendar.opens:
            continue

        date = pd.to_datetime(date)

        kbars = all_kbar[date:date+pd.Timedelta(days=1)]

        for ts in range(len(kbars)):

            #Entry Condition
            current_time = kbars.iloc[ts].name + pd.Timedelta(minutes=30)
            current_price = kbars['close'][ts]
            buy_price = kbars['open'][ts]

            if (
                #first condition
                position == 0 and 
                #OR condition
                #current_price/kbars.iloc[ts]['ma20']-1 < filter or 
                current_price > kbars[code].loc[str_current_time]['ma20']                        
            ):
                if (
                    #Entry condition
                    current_price > kbars[code].loc[str_current_time]['ma10']         
                ):
                    qty = 1 #改成金額決定股數
                    position = qty

                    entry_time = pd.Timestamp(date) + pd.Timedelta(hours=current_time.hour, minutes=current_time.minute, seconds=current_time.second)
                    entry_price = current_price

                    #Cost record
                    AC = current_price
                    EH = current_price
                    last_price = current_price
                    cost_after_add = current_price
                    
                    #Report
                    print( 'Entry: [{}] buy {} at {}'.format(current_time, codes, current_price))

            elif(
                position != 0
            ):
                #記錄進場後(不論有沒有進行加減碼)的 New Entry High 
                if current_price > EH:
                    EH = current_price

                add_position = 1
                cost_after_add = (AC * position + current_price * add_position) / (position + add_position)
                cv1 = EH * (1+Ratio_Drawdown1) / cost_after_add -1
                cv2 = EH * (1 + Ratio_Drawdown1) 
                

                #兩個加碼條件都滿足才加碼
                # 加碼條件1
                if cv1 > 0:
                    if cv2 > last_price:
                        qty = add_position

                        position += qty                         

                        #Cost record
                        AC = (AC * (position-add_position) + current_price * add_position) / position
                        EH = current_price
                        last_price = current_price

                            
                        #Report
                        print( 'Entry: [{}] Reach adding condition: {} and buy {} at {}'.format(current_time,  round(cv1, 5),
                                                                                        codes, current_price))
                        print( 'Now: [{}] buy {} for {} units in at average price of {} '.format(current_time, codes, position, round(AC,2)))

                    #出場條件1
                elif(
                    current_price <= EH * (1+Ratio_Drawdown2)
                ):
                    if position < 5:                       
                        exit_time = pd.Timestamp(date) + pd.Timedelta(hours=current_time.hour, minutes=current_time.minute, seconds=current_time.second)
                        exit_price = current_price

                        print('Exit: [{}] Reach condition1: {} and sell {} for {} units at {}'.format(current_time,
                                                                                                    round(EH * (1+Ratio_Drawdown1), 5),
                                                                                                    codes, position, current_price))
                        
                    elif(
                        position >= 5                 
                    ):
                        
                        exit_time = pd.Timestamp(date) + pd.Timedelta(hours=current_time.hour, minutes=current_time.minute, seconds=current_time.second)
                        exit_price = current_price
                        
                        print( 'Exit: [{}] Reach condition2: {} and sell {} for {} units at {}'.format(current_time, 
                                                                                                    round(EH * (1+Ratio_Drawdown2),5),
                                                                                                    codes, position, current_price))
                    break
                                                                 

            if entry_time and exit_time:
                transaction = pd.DataFrame([[date,
                                            codes,
                                            entry_time,
                                            AC,
                                            position ,
                                            exit_time,
                                            exit_price,
                                            position,
                                            exit_price,
                                            (exit_price-entry_price)*position*10]                                 
                                            ],
                                            columns = [
                                                '成交日期',
                                                '股票代號',
                                                '進場時間',
                                                '平均買進價格',
                                                '總買進口數',
                                                '賣出時間',
                                                '賣出價格',
                                                '總賣出口數',
                                                '賣出價格',
                                                '價差'])
                # Append this transaction to the overall transactions dataframe
                transactions = pd.concat([transactions, transaction], ignore_index=True)

                # Reset for the next trade
                AC = 0
                EH = 0
                position = 0
                entry_time = None
                exit_time = None

    # Return all transactions at the end
    return transactions
 
 